In [1]:
import feather
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

In [ ]:
%%time
df = feather.read_dataframe('/Volumes/transcend/大檔案/Taxi/NYT/nyc_taxi_data_raw.feather')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55423856 entries, 0 to 55423855
Data columns (total 7 columns):
fare_amount          float32
pickup_datetime      datetime64[ns, UTC]
pickup_longitude     float32
pickup_latitude      float32
dropoff_longitude    float32
dropoff_latitude     float32
passenger_count      uint8
dtypes: datetime64[ns, UTC](1), float32(5), uint8(1)
memory usage: 1.5 GB


## 資料前處理

### 刪除資料

In [4]:
# 遺失值
df.isnull().sum()

fare_amount            0
pickup_datetime        0
pickup_longitude       0
pickup_latitude        0
dropoff_longitude    376
dropoff_latitude     376
passenger_count        0
dtype: int64

In [3]:
# 去除遺失值
df = df.dropna()

In [9]:
df['passenger_count'].value_counts()

1      38337524
2       8175243
5       3929346
3       2432712
4       1178852
6       1174647
0        195040
208          64
9            23
7            15
8             9
129           2
51            1
49            1
34            1
Name: passenger_count, dtype: int64

In [4]:
# 限制乘客範圍為1~6
df = df[(df['passenger_count']==1)|(df['passenger_count']==2)|(df['passenger_count']==3)|(df['passenger_count']==4)|(df['passenger_count']==5)|(df['passenger_count']==6)]

In [5]:
# 去除價錢小於0
df = df[df['fare_amount']>0]

In [6]:
# 限制經緯度範圍
df = df[(df['pickup_longitude']>=-74.5)&(df['pickup_longitude']<=-72.8)&(df['pickup_latitude']>=40.5)&(df['pickup_latitude']<=41.8)&(df['dropoff_longitude']>=-74.5)&(df['dropoff_longitude']<=-72.8)&(df['dropoff_latitude']>=40.5)&(df['dropoff_latitude']<=41.8)]

In [7]:
# 9筆價格異常高的誇張
df[df['fare_amount']>1000]

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance,year,month,weekday,hour
18133357,2010.900024,2010-03-21 10:03:00+00:00,-73.986267,40.774845,-73.971542,40.743950,1,3.649243,2010,3,Sunday,10
18393336,61550.859375,2013-08-20 11:52:00+00:00,-73.967712,40.755966,-73.945572,40.747665,1,2.084641,2013,8,Tuesday,11
22142518,3130.300049,2015-03-09 15:28:00+00:00,-74.005417,40.664413,-74.005417,40.664413,1,0.000000,2015,3,Monday,15
30060980,2004.500000,2010-03-20 12:24:00+00:00,-73.963135,40.777855,-73.975433,40.782146,1,1.142337,2010,3,Saturday,12
36942979,2023.699951,2010-03-21 14:43:00+00:00,-73.790054,40.647064,-73.917976,40.656940,1,10.874598,2010,3,Sunday,14
38401219,9000.410156,2010-02-19 17:10:00+00:00,-73.993744,40.746105,-73.989807,40.738976,1,0.858729,2010,2,Friday,17
40017986,93963.359375,2015-02-24 08:57:00+00:00,-73.994461,40.712353,-74.012543,40.702904,2,1.853598,2015,2,Tuesday,8
40167920,75747.023438,2015-05-03 09:35:00+00:00,-73.967949,40.792355,-73.990898,40.738976,1,6.236368,2015,5,Sunday,9
43891292,1564.500000,2010-02-20 17:38:00+00:00,-73.937782,40.758255,-73.937752,40.758232,1,0.003620,2010,2,Saturday,17


In [8]:
df = df[df['fare_amount']<1000]

In [6]:
# 有582996筆距離0的資料，但test裡也有距離為0，所以不刪除
df[df['distance']==0].head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance,year,month,weekday,hour
102,52.0,2009-03-25 00:08:00+00:00,-74.035835,40.747318,-74.035835,40.747318,1,0.0,2009,3,Wednesday,0
187,6.5,2014-01-08 21:55:00+00:00,-73.998489,40.726303,-73.998489,40.726303,1,0.0,2014,1,Wednesday,21
264,7.5,2012-08-25 01:53:00+00:00,-73.995895,40.746452,-73.995895,40.746452,1,0.0,2012,8,Saturday,1
283,6.9,2009-12-14 12:33:00+00:00,-73.982430,40.745747,-73.982430,40.745747,1,0.0,2009,12,Monday,12
385,12.0,2014-03-12 18:12:00+00:00,-73.844902,40.736317,-73.844902,40.736317,1,0.0,2014,3,Wednesday,18


In [10]:
df = df.reset_index(drop=True)

In [11]:
len(df)

54054801

### 新增欄位

In [9]:
import geopy.distance

In [10]:
dis = []
for i in tqdm_notebook(range(len(df))):
    try:
        dis.append(geopy.distance.vincenty((df['pickup_latitude'][i],df['pickup_longitude'][i]),(df['dropoff_latitude'][i],df['dropoff_longitude'][i])).km)
    except:
        print(i)
        break
df['distance'] = pd.DataFrame(dis)

/Users/andy/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.


In [12]:
df['year'] = df['pickup_datetime'].dt.year
df['month'] = df['pickup_datetime'].dt.month
df['weekday'] = df['pickup_datetime'].dt.weekday
df['weekday'] = df['weekday'].replace([0, 1, 2, 3, 4, 5, 6], ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
df['hour'] = df['pickup_datetime'].dt.hour

In [12]:
df.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance,year,month,weekday,hour
0,4.5,2009-06-15 17:26:00+00:00,-73.844315,40.721317,-73.841614,40.712276,1,1.029579,2009,6,Monday,17
1,16.9,2010-01-05 16:52:00+00:00,-74.016045,40.711304,-73.979271,40.782005,1,8.443307,2010,1,Tuesday,16
2,5.7,2011-08-18 00:35:00+00:00,-73.982735,40.761269,-73.991241,40.750561,2,1.389240,2011,8,Thursday,0
3,7.7,2012-04-21 04:30:00+00:00,-73.987129,40.733143,-73.991570,40.758091,1,2.795730,2012,4,Saturday,4
4,5.3,2010-03-09 07:51:00+00:00,-73.968094,40.768009,-73.956657,40.783764,1,1.998262,2010,3,Tuesday,7


In [13]:
%%time
# Save into feather format, about 3.9Gb.
df.to_feather('/Volumes/transcend/大檔案/Taxi/NYT/nyc_taxi_data_raw_clean_origin.feather')

CPU times: user 9.91 s, sys: 19.2 s, total: 29.1 s
Wall time: 1min 16s


### 需要的欄位

In [14]:
df2 = df[['fare_amount','passenger_count','distance','year','month','weekday','hour']]

In [15]:
df2.head()

,fare_amount,passenger_count,distance,year,month,weekday,hour
0,4.5,1,1.029579,2009,6,Monday,17
1,16.9,1,8.443307,2010,1,Tuesday,16
2,5.7,2,1.389240,2011,8,Thursday,0
3,7.7,1,2.795730,2012,4,Saturday,4
4,5.3,1,1.998262,2010,3,Tuesday,7


In [16]:
%%time
# Save into feather format, about 2.6Gb.
df2.to_feather('/Volumes/transcend/大檔案/Taxi/NYT/nyc_taxi_data_raw_clean.feather')

CPU times: user 6.54 s, sys: 12.7 s, total: 19.2 s
Wall time: 56.4 s
